## Import necessary modules
Run this cell before running any other cells

In [16]:
%load_ext autoreload
%autoreload 2

from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import time
import numpy as np

LOG.propagate = False

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Printing and Logging
## Printing
You can use the **print()** function in Python to print messages to the screen. <br>
The message can be a string, or any other object, the object will be converted into a string before it is written to the screen. <br>

## Logging
You could use the logging module that is setup in *utils.py*. <br>
It prints to both your screen (standard output) as well as to log files (*ble.log*) in the *logs* directory. <br>
This is the recommended way to output messages, since the log files can help with debugging. <br>
The logging module also provides different log levels as shown below, each formatted with a different color for increased visibility. <br>

__**NOTE**__: You may notice that the DEBUG message is not printed to the screen but is printed in the log file. This is because the logging level for the screen is set to INFO and for the file is set to DEBUG. You can change the default log levels in *utils.py* (**STREAM_LOG_LEVEL** and **FILE_LOG_LEVEL**). 

## Formatting output
To format your strings, you may use %-formatting, str.format() or f-strings. <br>
The most "pythonic" way would be to use f-strings. [Here](https://realpython.com/python-f-strings/) is a good tutorial on f-strings. <br>

In [17]:
LOG.debug("debug")
LOG.info("info")
LOG.warning("warning")
LOG.error("error")
LOG.critical("critical")

2023-02-15 00:36:01,145 | INFO     |: info
2023-02-15 00:36:01,146 | WARNING  |: warning
2023-02-15 00:36:01,148 | ERROR    |: error
2023-02-15 00:36:01,149 | CRITICAL |: critical


<hr>

# BLE
## ArtemisBLEController
The class **ArtemisBLEController** (defined in *ble.py*) provides member functions to handle various BLE operations to send and receive data to/from the Artemis board, provided the accompanying Arduino sketch is running on the Artemis board. <br>

<table align="left">
     <tr>
        <th style="text-align: left; font-size: medium">Member Functions</th>
        <th style="text-align: left; font-size: medium">Description</th style="text-align: left">
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">reload_config()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Reload changes made in <em>connection.yaml.</em></span></th style="text-align: left">
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">connect()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Connect to the Artemis board, whose MAC address is specified in <em>connection.yaml</em>.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">disconnect()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Disconnect from the Artemis board.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">is_connected()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Return a boolean indicating whether your controller is connected to the Artemis board or not.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">send_command(cmd_type, data)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Send the command <strong>cmd_type</strong> (integer) with <strong>data</strong> (string) to the Artemis board.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">receive_float(uuid) <br> receive_string(uuid) <br> receive_int(uuid)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Read the GATT characteristic (specified by its <strong>uuid</strong>) of type float, string or int. <br> The type of the GATT
            characteristic is determined by the classes BLEFloatCharacteristic, BLECStringCharacteristic or
            BLEIntCharacteristic in the Arduino sketch.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">start_notify(uuid, notification_handler)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Activate notifications on the GATT characteristic (specified by its <strong>uuid</strong>). <br> <strong>notification_handler</strong> is a
            function callback which must accept two inputs; the first will be a uuid string object and the second will
            be the bytearray of the characteristic value.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">bytearray_to_float(byte_array) <br> bytearray_to_string(byte_array) <br> bytearray_to_int(byte_array)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Convert the <strong>bytearray</strong> to float, string or int, respectively. <br> You may use these functions inside your
            notification callback function.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">stop_notify(uuid)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Stop notifications on the GATT characteristic (specified by its <strong>uuid</strong>).</span></th>
    </tr>
</table>

<table align="left">
     <tr>
        <th style="text-align: left; font-size: medium">Member Variables</th>
        <th style="text-align: left; font-size: medium">Description</th style="text-align: left">
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">uuid</span></th>
        <th style="text-align: left"><span style="font-weight: normal">A dictionary that stores the UUIDs of the various characteristics specified in <em>connection.yaml</em>.</span></th>
    </tr>
</table>

## Configuration
- The MAC address, Service UUID and GATT characteristic UUIDs are defined in the file: *connection.yaml*.
- They should match the UUIDs used in the Arduino sketch.
- The artemis board running the base code should display its MAC address in the serial monitor.
- Update the **artemis_address** in *connection.yaml*, accordingly.
- Make sure to call **ble.reload_config()** or **get_ble_controller()** (which internally calls **reload_config()**) after making any changes to your configuration file.

<hr>

In the below cell, we create an **ArtemisBLEController** object using **get_ble_controller()** (defined in *ble.py*), which creates and/or returns a single instance of **ArtemisBLEController**. <br>
<span style="color:rgb(240,50,50)"> __NOTE__: Do not use the class directly to instantiate an object. </span><br>

In [18]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

2023-02-15 00:36:02,627 | INFO     |: Looking for Artemis Nano Peripheral Device: c0:07:1d:96:09:44
2023-02-15 00:36:07,114 | INFO     |: Connected to c0:07:1d:96:09:44


## Receive data from the Artemis board

The cell below shows examples of reading different types (as defined in the Arduino sketch) of GATT characteristics.

In [19]:
# Read a float GATT Charactersistic
f = ble.receive_float(ble.uuid['RX_FLOAT'])
print(f)

3.5


In [20]:
# Read a string GATT Charactersistic
s = ble.receive_string(ble.uuid['RX_STRING'])
print(s)

[->9.0<-]


## Send a command to the Artemis board
Send the PING command and read the reply string from the string characteristic RX_STRING. <br>
__NOTE__: The **send_command()** essentially sends a string data to the GATT characteristic (TX_CMD_STRING). The GATT characteristic in the Arduino sketch is of type BLECStringCharacteristic.

In [21]:
ble.send_command(CMD.PING, "")

In [22]:
s = ble.receive_string(ble.uuid['RX_STRING'])
print(s)

PONG


The cell below shows an example of the SEND_TWO_INTS command. <br> The two values in the **data** are separated by a delimiter "|". <br>
Refer Lab 2 documentation for more information on the command protocol.

In [23]:
ble.send_command(CMD.SEND_TWO_INTS, "2|-6")

The Artemis board should print the two integers to the serial monitor in the ArduinoIDE. 

#### ECHO

In [24]:
ble.send_command(CMD.ECHO, "Hello!")
s = ble.receive_string(ble.uuid['RX_STRING'])
print(s)

Artemis Says: Hello!


#### GET_TIME_MILLIS

In [25]:
ble.send_command(CMD.GET_TIME_MILLIS,"")
s = ble.receive_string(ble.uuid['RX_STRING'])
print(s)

T:34872


#### Notification Handler

In [26]:
def get_millis_handler(uuid, byte_array):
    time = ble.bytearray_to_string(byte_array)
    time = time[2:]
    print(time)
    
def get_temp_handler(uuid, byte_array):
    temp = ble.bytearray_to_string(byte_array).split('|')
    count = 1
    for i in range(1,6) :
        print (i, ": ", temp[i-1])       

count = 1
def string_handler(uuid, byte_array):
    global count
    tempRapid = ble.bytearray_to_string(byte_array)
    
    if tempRapid == "END" :
        count = 1
        
    elif "C" not in tempRapid:
        print(tempRapid[2:])
    
    else :         
        print(count, ": ", tempRapid)
        count+=1
    
        
    
ble.start_notify(ble.uuid['RX_STRING'], string_handler)

    

In [27]:
ble.send_command(CMD.GET_TIME_MILLIS,"")
ble.send_command(CMD.GET_TIME_MILLIS,"")

37681
37802


In [28]:
ble.send_command(CMD.GET_TEMP_5s,"")
ble.send_command(CMD.GET_TEMP_5s,"")

1 :  T:39416 C:23.556
2 :  T:40418 C:23.481
3 :  T:41419 C:23.68
4 :  T:42420 C:23.593
5 :  T:43421 C:24.6
1 :  T:43425 C:23.443
2 :  T:44426 C:23.518
3 :  T:45427 C:23.781
4 :  T:46428 C:23.593
5 :  T:47429 C:23.293


In [30]:
ble.send_command(CMD.GET_TEMP_5s_RAPID,"")

1 :  T:59920 C:23.106
2 :  T:59923 C:23.218
3 :  T:59926 C:23.406
4 :  T:59933 C:23.406
5 :  T:59978 C:23.631
6 :  T:59988 C:23.331
7 :  T:59993 C:23.181
8 :  T:60040 C:23.518
9 :  T:60044 C:23.443
10 :  T:60047 C:23.406
11 :  T:60050 C:23.368
12 :  T:60100 C:22.318
13 :  T:60103 C:22.955
14 :  T:60106 C:22.993
15 :  T:60109 C:23.443
16 :  T:60158 C:23.143
17 :  T:60165 C:23.743
18 :  T:60168 C:23.631
19 :  T:60176 C:23.31
20 :  T:60221 C:23.443
21 :  T:60225 C:23.31
22 :  T:60228 C:22.730
23 :  T:60230 C:23.406
24 :  T:60283 C:23.593
25 :  T:60286 C:23.856
26 :  T:60291 C:23.968
27 :  T:60294 C:22.918
28 :  T:60339 C:23.518
29 :  T:60343 C:23.406
30 :  T:60346 C:23.181
31 :  T:60349 C:23.181
32 :  T:60399 C:22.993
33 :  T:60402 C:23.218
34 :  T:60405 C:23.668
35 :  T:60408 C:23.31
36 :  T:60457 C:22.993
37 :  T:60464 C:23.518
38 :  T:60467 C:23.143
39 :  T:60473 C:23.893
40 :  T:60522 C:23.481
41 :  T:60529 C:23.68
42 :  T:60532 C:23.181
43 :  T:60534 C:22.318
44 :  T:60578 C:23.143
4

In [25]:
ble.stop_notify(ble.uuid['RX_STRING'])

## Disconnect

In [59]:
# Disconnect
ble.disconnect()

Exception: Not connected to a BLE device